In [ ]:
import mne
from mne.preprocessing import ICA
from  mne_icalabel import label_components
from pathlib import Path

In [ ]:
sample_data_raw_file = "/Users/podlaskijacek/Documents/HMM-Analysis/s1_02_preproc-raw.fif"
raw = mne.io.read_raw_fif(sample_data_raw_file)

raw.load_data()
print(raw.ch_names)

In [ ]:
#Plotting selected channels
artifact_picks = mne.pick_channels_regexp(raw.ch_names, regexp=r"(MZ.)") #specify the prefix of plotted channels
raw.plot(order=artifact_picks, n_channels=len(artifact_picks), show_scrollbars=False)

In [ ]:
ica = ICA(
    n_components=80,
    max_iter="auto",
    method="infomax",
    random_state=97,
    fit_params=dict(extended=True),
)
ica.fit(raw)
ica
ica.plot_sources(raw, show_scrollbars=False, show=True)

In [ ]:
ica.plot_sources(raw, show_scrollbars=False, show=True)
ica.plot_components()
#ica.plot_overlay(raw, exclude=[0], picks="meg") #blinks

In [ ]:
ic_labels = label_components(raw, ica, method="megnet")
print(ic_labels["labels"])

In [ ]:
labels = ic_labels["labels"]
exclude_idx = [idx for idx, label in enumerate(labels) if label not in ["brain/other"]]
print(f"Excluding these ICA components: {exclude_idx}")
ica.plot_properties(raw, picks=exclude_idx)

In [ ]:
#Plotting before and after ICA
reconst_raw = raw.copy()
ica.apply(reconst_raw, exclude=exclude_idx)

raw.plot()
reconst_raw.plot()
del reconst_raw

In [ ]:
preproc_file = Path(f"/Users/podlaskijacek/Documents/HMM-Analysis/s1_03_ICA.fif") #.fif file format
preproc_file.parent.mkdir(parents=True, exist_ok=True)
raw.save(preproc_file, overwrite=True)
raw.load_data()